In [ ]:
import numpy as np
from PIL import Image
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
import scipy.io as spio
import os

In [ ]:
# Load the images, keypoints, and descriptors
data_path = Path(os.getcwd()) / 'isr_wall'

keypoints_fpaths = sorted((data_path / 'keypoints').glob('*.mat'),
                     key=lambda x: int(x.stem.split('_')[1])
                    )

images_fpaths = sorted((data_path / 'images').glob('*.jpg'),
                     key=lambda x: int(x.stem.split('_')[1])
                    )

keypoints = [spio.loadmat(fpath)['kp'] for fpath in keypoints_fpaths]
descriptors = [spio.loadmat(fpath)['desc'] for fpath in keypoints_fpaths]
images = [np.array(Image.open(fpath)) for fpath in images_fpaths]

In [ ]:
# Function for keypoint matching
def find_idx_matches(descs_img_1, descs_img_2):
    matches_idx = []
    for src_idx, desc in enumerate(descs_img_1):
        desc_diffs = descs_img_2 - desc
        norms = np.linalg.norm(desc_diffs, axis=1)
        best_dest_idx = np.argmin(norms)
        matches_idx.append((src_idx, best_dest_idx))
    return matches_idx

# Function for finding keypoint matches
def find_keypoint_matches(descs_1, descs_2, keypoints_1, keypoints_2):
    matches_idx = find_idx_matches(descs_1, descs_2)
    matches = []

    for src_idx, dest_idx in matches_idx:
        src_kp = keypoints_1[src_idx]
        dest_kp = keypoints_2[dest_idx]
        matches.append([src_kp[0], src_kp[1], dest_kp[0], dest_kp[1]])
    
    return np.array(matches)

# Homography computation
def compute_homography(src_pts, dst_pts):
    num_points = src_pts.shape[0]
    A = []
    
    for i in range(num_points):
        x, y = src_pts[i]
        x_prime, y_prime = dst_pts[i]
        A.append([-x, -y, -1, 0, 0, 0, x * x_prime, y * x_prime, x_prime])
        A.append([0, 0, 0, -x, -y, -1, x * y_prime, y * y_prime, y_prime])
    
    A = np.array(A)
    _, _, Vt = np.linalg.svd(A)
    h = Vt[-1]
    H = h.reshape(3, 3)
    return H / H[-1, -1]

"""" RANSAC

Given N samples 

1) Randomly draw n samples
2) Estimate Model
3) Select inliers by computing the error and apply threshold

repeate 1), 2), 3) K times

4) Select Model with the most inliers
5) reestimade the model with all inliers

"""
def compute_ransac_homography(src_pts, dst_pts, threshold=5.0, K=1000, P=0.99):
    """Compute homography using RANSAC."""
    best_H = None
    max_inliers = 0
    n_points = len(src_pts)
    best_inlier_src, best_inlier_dst = None, None
    
    for _ in range(K):
        # 1) Randomly draw 4 samples
        indices = np.random.choice(n_points, 4, replace=False)
        sample_src = src_pts[indices]
        sample_dst = dst_pts[indices]

        # 2) Estimate Model
        try:
            H = compute_homography(sample_src, sample_dst)
        except ValueError:
            # Skip degenerate configurations
            continue

        # 3) Select inliers by computing the error and apply threshold
        # Project all points
        src_homog = np.hstack((src_pts, np.ones((n_points, 1))))  # Convert to homogeneous
        projected_pts = (H @ src_homog.T).T
        projected_pts /= projected_pts[:, 2:3]  # Normalize to inhomogeneous
        distances = np.linalg.norm(projected_pts[:, :2] - dst_pts, axis=1)
        inliers = distances < threshold

        # Update best homography if more inliers are found
        num_inliers = np.sum(inliers)
        if num_inliers > max_inliers:
            max_inliers = num_inliers
            best_H = H
            best_inlier_src = src_pts[inliers]
            best_inlier_dst = dst_pts[inliers]

    # 5) Re-estimate model with all inliers
    if max_inliers > 0:
        best_H = compute_homography(best_inlier_src, best_inlier_dst)
    
    return best_inlier_src, best_inlier_dst, best_H

def compute_homographies(num_images, keypoints, descriptors):

    #Match keypoints and compute homographies for each consecutive pair of images.
    homographies = []

    for i in range(num_images - 1):
        # Find matches between keypoints
        kp_matches = find_keypoint_matches(descriptors[i], descriptors[i+1], keypoints[i], keypoints[i+1])
        src_pts = kp_matches[:, :2]
        dst_pts = kp_matches[:, 2:]
        
        # Compute homography using RANSAC
        _, _, H = compute_ransac_homography(src_pts, dst_pts, threshold=5.0, K=10000, P=0.99)

        homographies.append(H)

    return homographies

#homographies = compute_homographies(len(images), keypoints, descriptors)

In [ ]:
def compute_cumulative_homographies(num_images, keypoints, descriptors, ref_index=0):
    """Compute homographies between each image and a reference image."""
    homographies = [np.eye(3)]  # Homography from the reference image to itself is identity
    
    for i in range(1, num_images):
        # Find matches between keypoints of consecutive images
        kp_matches = find_keypoint_matches(descriptors[i-1], descriptors[i], keypoints[i-1], keypoints[i])
        src_pts = kp_matches[:, :2]
        dst_pts = kp_matches[:, 2:]
        
        # Compute homography between consecutive images using RANSAC
        _, _, H_i_to_i_minus_1 = compute_ransac_homography(src_pts, dst_pts, threshold=5.0, K=1000, P=0.99)
        
        homographies.append(homographies[-1] @ np.linalg.inv(H_i_to_i_minus_1))
    
    return homographies

In [ ]:
def compute_cumulative_homographies_optimized(num_images, keypoints, descriptors, ref_index=0, inlier_threshold=30):
    """Compute cumulative homographies with an optimization to use direct transformations when possible."""
    homographies = [np.eye(3)]  # Homography from the reference image to itself is identity

    for i in range(1, num_images):
        print(f"Computing transformation for image {i + 1} ...")
        
        # Find matches between current image and reference image
        kp_matches_ref = find_keypoint_matches(descriptors[ref_index], descriptors[i], keypoints[ref_index], keypoints[i])
        src_pts_ref = kp_matches_ref[:, :2]
        dst_pts_ref = kp_matches_ref[:, 2:]
        
        # Compute homography between reference and current image using RANSAC
        inlier_src_pts, inlier_dst_pts, H_ref_to_i = compute_ransac_homography(src_pts_ref, dst_pts_ref, threshold=5.0, K=1000, P=0.99)
        
        # Count the number of inliers (size of the inlier points)
        num_inliers = inlier_src_pts.shape[0]
        print(f"Inliers between {ref_index} and {i + 1} is : {num_inliers}.")

        if num_inliers >= inlier_threshold:
            print(f"Using direct transformation.")
            # Use direct transformation if enough inliers are found
            homographies.append(H_ref_to_i)
        else:
            # Fallback to cumulative chaining
            print(f"Fallback to cumulative chaining.")
            kp_matches = find_keypoint_matches(descriptors[i - 1], descriptors[i], keypoints[i - 1], keypoints[i])
            src_pts = kp_matches[:, :2]
            dst_pts = kp_matches[:, 2:]

            _, _, H_i_to_i_minus_1 = compute_ransac_homography(src_pts, dst_pts, threshold=5.0, K=1000, P=0.99)
            
            # Cumulative homography
            homographies.append(homographies[-1] @ np.linalg.inv(H_i_to_i_minus_1))


        ##THIS IS JUST FOR VISUALIZATION

        # Visualization of keypoints and inliers
        print(f"Visualizing keypoints and inliers between image 1 and image {i + 1}...")

        # Apply the homography to the points from the reference image
        src_homog_ref = np.hstack((inlier_src_pts, np.ones((inlier_src_pts.shape[0], 1))))  # Convert to homogeneous
        projected_pts = (H_ref_to_i @ src_homog_ref.T).T
        projected_pts /= projected_pts[:, 2:3]  # Normalize to get (x, y) coordinates

        # Plot the images side by side
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))

        # Load images
        ref_image = cv2.imread("isr_wall/images/img_0001.jpg")  # Load reference image (image 1)
        curr_image = cv2.imread(f"isr_wall/images/img_000{i + 1}.jpg")  # Load the current image (image i)

        # Convert images to RGB for matplotlib display
        ref_image_rgb = cv2.cvtColor(ref_image, cv2.COLOR_BGR2RGB)
        curr_image_rgb = cv2.cvtColor(curr_image, cv2.COLOR_BGR2RGB)

        # Plot the reference image with keypoints and inliers
        ax[0].imshow(ref_image_rgb)
        ax[0].scatter(src_pts_ref[:, 0], src_pts_ref[:, 1], color='g', s=5, label='Keypoints on Ref Image')
        ax[0].scatter(inlier_src_pts[:, 0], inlier_src_pts[:, 1], color='r', s=5, label='Inliers on Ref Image')
        ax[0].set_title(f"Reference Image (Image 1)")
        ax[0].legend()

        # Plot the current image with projected inliers
        ax[1].imshow(curr_image_rgb)
        ax[1].scatter(dst_pts_ref[:, 0], dst_pts_ref[:, 1], color='g', s=5, label='Keypoints on Ref Image')
        ax[1].scatter(projected_pts[:, 0], projected_pts[:, 1], color='r', s=5, label='Inliers on Transformed Image')
        ax[1].set_title(f"Transformed Image {i + 1}")
        ax[1].legend()

        plt.show()



    print(f"Finshed calculating transformations")
    return homographies


cumulative_homographies = compute_cumulative_homographies_optimized(len(images), keypoints, descriptors, ref_index=0)